In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/hjj/20206/manjora/cpx222/train/'

test_path='/home/hjj/20206/manjora/cpx222/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
#series_num=3 #input of D


In [3]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=round(np.float64(float(energy_string)),5)
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [4]:
global extend_num, move_num

In [5]:
extend_num=1000

In [6]:
move_num=get_energy(train_path+'00000/')
print(move_num)

-122.69044


In [7]:
#base_pxrd_s=mg.Structure.from_file('/home/mii/Desktop/cpx222/train/00000/CONTCAR')
#base_pxrd=patt_xrd.get_pattern(base_pxrd_s)

In [8]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###
def get_xrdmat3(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    #xrd_data4.y=xrd_data4.y-base_pxrd.y
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i] > 0.1 and xrd_data4.y[i] < 70:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.sqrt(1/100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4
###
###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat3(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [9]:
class GCNNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GCNNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

In [10]:
G1= GCNNet()
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.0001)
#mat_Gl=[]    
#pre_gg=[]

In [11]:
trainfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        trainfile.append(ieach)
i=0
loss_set=[]
for path_ in trainfile:
    E_Gibbs=[]
    try:
        total_energy=get_energy(path_)
        tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
    except:
        print(path_)
            
    E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    g_in=[]
           
     
    try:
        tomgS=tomgStructure(path_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)    
    
    G1_loss=torch.abs(torch.mean(Gout-float(tt_energy)))
    

    opt_G1.zero_grad()
    G1_loss.backward()
    opt_G1.step()
    
    i += 1
    loss_set.append(G1_loss)
    print(i,": ",G1_loss)

/home/hjj/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pymatgen/io/cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


1 :  tensor(3.69493958, grad_fn=<AbsBackward>)
2 :  tensor(9.25461852, grad_fn=<AbsBackward>)
3 :  tensor(3.80833784, grad_fn=<AbsBackward>)
4 :  tensor(1.56723135, grad_fn=<AbsBackward>)
5 :  tensor(1.53950041, grad_fn=<AbsBackward>)
6 :  tensor(0.14652745, grad_fn=<AbsBackward>)
7 :  tensor(12.20708922, grad_fn=<AbsBackward>)
8 :  tensor(5.50980173, grad_fn=<AbsBackward>)
9 :  tensor(7.49188625, grad_fn=<AbsBackward>)
10 :  tensor(1.57723525, grad_fn=<AbsBackward>)
11 :  tensor(0.29384186, grad_fn=<AbsBackward>)
12 :  tensor(10.08099401, grad_fn=<AbsBackward>)
13 :  tensor(5.34886498, grad_fn=<AbsBackward>)
14 :  tensor(1.24860857, grad_fn=<AbsBackward>)
15 :  tensor(11.67163208, grad_fn=<AbsBackward>)
16 :  tensor(12.04178738, grad_fn=<AbsBackward>)
17 :  tensor(15.12244649, grad_fn=<AbsBackward>)
18 :  tensor(1.05621284, grad_fn=<AbsBackward>)
19 :  tensor(9.84610605, grad_fn=<AbsBackward>)
20 :  tensor(5.00645253, grad_fn=<AbsBackward>)
21 :  tensor(4.35150657, grad_fn=<AbsBackwar

170 :  tensor(1.44537756, grad_fn=<AbsBackward>)
171 :  tensor(5.22877829, grad_fn=<AbsBackward>)
172 :  tensor(3.09894921, grad_fn=<AbsBackward>)
173 :  tensor(3.19290879, grad_fn=<AbsBackward>)
174 :  tensor(3.65660639, grad_fn=<AbsBackward>)
175 :  tensor(4.32627196, grad_fn=<AbsBackward>)
176 :  tensor(7.65090325, grad_fn=<AbsBackward>)
177 :  tensor(5.96799643, grad_fn=<AbsBackward>)
178 :  tensor(3.22340053, grad_fn=<AbsBackward>)
179 :  tensor(3.94757926, grad_fn=<AbsBackward>)
180 :  tensor(6.33551129, grad_fn=<AbsBackward>)
181 :  tensor(0.05178084, grad_fn=<AbsBackward>)
182 :  tensor(5.93256969, grad_fn=<AbsBackward>)
183 :  tensor(14.78808302, grad_fn=<AbsBackward>)
184 :  tensor(5.56884459, grad_fn=<AbsBackward>)
185 :  tensor(5.78280101, grad_fn=<AbsBackward>)
186 :  tensor(3.60496733, grad_fn=<AbsBackward>)
187 :  tensor(3.01003537, grad_fn=<AbsBackward>)
188 :  tensor(1.02625857, grad_fn=<AbsBackward>)
189 :  tensor(2.08387878, grad_fn=<AbsBackward>)
190 :  tensor(6.037

338 :  tensor(3.50529587, grad_fn=<AbsBackward>)
339 :  tensor(1.53936212, grad_fn=<AbsBackward>)
340 :  tensor(3.62363490, grad_fn=<AbsBackward>)
341 :  tensor(15.79267303, grad_fn=<AbsBackward>)
342 :  tensor(1.90442571, grad_fn=<AbsBackward>)
343 :  tensor(0.03487194, grad_fn=<AbsBackward>)
344 :  tensor(1.52274728, grad_fn=<AbsBackward>)
345 :  tensor(5.27444915, grad_fn=<AbsBackward>)
346 :  tensor(7.35125985, grad_fn=<AbsBackward>)
347 :  tensor(8.91008621, grad_fn=<AbsBackward>)
348 :  tensor(3.61922922, grad_fn=<AbsBackward>)
349 :  tensor(6.80723075, grad_fn=<AbsBackward>)
350 :  tensor(4.92463022, grad_fn=<AbsBackward>)
351 :  tensor(0.52633087, grad_fn=<AbsBackward>)
352 :  tensor(0.50596500, grad_fn=<AbsBackward>)
353 :  tensor(1.56129306, grad_fn=<AbsBackward>)
354 :  tensor(4.73271803, grad_fn=<AbsBackward>)
355 :  tensor(2.23567050, grad_fn=<AbsBackward>)
356 :  tensor(0.04311872, grad_fn=<AbsBackward>)
357 :  tensor(4.55325912, grad_fn=<AbsBackward>)
358 :  tensor(10.73

In [12]:
torch.save(G1.state_dict(),"/home/hjj/Desktop/G1_GCN-526.pkl") 

In [13]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
start=time.time()        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

651.5525207519531


In [ ]:
315.1618971824646

In [14]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

0.005746552301173864
4.437478543631989e-05


In [ ]:
print(np.sqrt(np.asarray(MSEset).mean()))

In [15]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [16]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

0.004843350861039625
3.3811323328157466e-05


In [ ]:

np.sqrt(2.463629398696308e-05)